# Pre-encoding a dataset for DALLE·mini

This notebook shows how to pre-encode images to token sequences using JAX, VQGAN and a dataset in the [`webdataset` format](https://webdataset.github.io/webdataset/).

Adapt it to your own dataset and image encoder.

(Changes) Cuda support added.

At the end you should have a dataset of pairs:
* a caption defined as a string
* an encoded image defined as a list of int.

In [ ]:
import os
cuda_lib_path = os.environ['CONDA_PREFIX'] + '/lib'
os.environ['LD_LIBRARY_PATH'] = (
    cuda_lib_path
    if 'LD_LIBRARY_PATH' not in os.environ else
    cuda_lib_path + ":" + os.environ['LD_LIBRARY_PATH']
)
print(os.environ['LD_LIBRARY_PATH'])


# https://github.com/borisdayma/dalle-mini/issues/280
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.8"


In [ ]:
from tqdm.notebook import tqdm

import torchvision.transforms as T

import webdataset as wds

import jax
import braceexpand
from pathlib import Path

## Configuration Parameters

In [ ]:
# shards = "my_images/shard-{0000..0008}.tar"  # defined using braceexpand format as used by webdataset
shards = "/home/ubuntu/craiyon/data/cc12m.tar"
encoded_output = Path("encoded_data")  # where we will save our encoded data

VQGAN_REPO, VQGAN_COMMIT_ID = (
    "dalle-mini/vqgan_imagenet_f16_16384",
    # "85eb5d3b51a1c62a0cc8f4ccdee9882c0d0bd384", # broken commit
    "e93a26e7707683d349bf5d5c41c5b0ef69b677a9",
)

# good defaults for a TPU v3-8
batch_size = 16  # Per device
num_workers = 8  # For parallel processing
total_bs = batch_size * jax.device_count()  # You can use a smaller size while testing
save_frequency = 16  # Number of batches to create a new file (180MB for f16 and 720MB for f8 per file)

In [ ]:
shards = list(
    braceexpand.braceexpand(shards)
)  # better display for tqdm with known length

## Load data

We load data using `webdataset`.

In [ ]:
ds = (
    wds.WebDataset(shards, handler=wds.warn_and_continue)
    .decode("rgb", handler=wds.warn_and_continue)
    .map_dict(jpg=T.ToTensor())
    .map_dict(jpg=T.Resize((256, 256)))
    .map_dict(jpg=lambda x: x.permute(1, 2, 0))
    .to_tuple("jpg", "txt")  # assumes image is in `jpg` and caption in `txt`
    .batched(total_bs)  # load in batch per worker (faster)
)

Note:
* you can also shuffle shards and items using `shardshuffle` and `shuffle` if necessary.
* you may need to resize images in your pipeline (with `map_dict` for example), we assume they are already set to 256x256.
* you can also filter out some items using `select`.

We can now inspect our data.

In [ ]:
%%time
images, captions = next(iter(ds))

In [ ]:
images.shape, images[0].dtype, len(captions)

In [ ]:
import torch
T.ToPILImage(mode="RGB")(images[0].permute(2, 0, 1))

Finally we create our dataloader.

In [ ]:
dl = (
    wds.WebLoader(ds, batch_size=None, num_workers=8).unbatched().batched(total_bs)
)  # avoid partial batch at the end of each worker

## Image encoder

We'll use a VQGAN trained with Taming Transformers and converted to a JAX model.

In [ ]:
from vqgan_jax.modeling_flax_vqgan import VQModel
from flax.jax_utils import replicate

vqgan, vqgan_params = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID, _do_init=False)

vqgan_params = replicate(vqgan_params)

## Encoding

Encoding is really simple using `shard` to automatically distribute batches across devices and `pmap`.

In [ ]:
from flax.training.common_utils import shard
from functools import partial


@partial(jax.pmap, axis_name="batch")
def p_encode(batch, params):
    # Not sure if we should `replicate` params, does not seem to have any effect
    print(batch.shape)
    _, indices = vqgan.encode(batch, params=params)
    # vqgan.encode(indices, params=params)
    return indices

In [ ]:
import pandas as pd


def encode_dataset(dataloader, output_dir, save_frequency):
    output_dir.mkdir(parents=True, exist_ok=True)
    all_captions = []
    all_encoding = []
    n_file = 1
    for idx, (images, captions) in enumerate(tqdm(dataloader)):
        images = images.numpy()
        n = len(images) // 8 * 8
        if n != len(images):
            # get the max number of images we can (multiple of 8)
            print(f"Different sizes {n} vs {len(images)}")
            images = images[:n]
            captions = captions[:n]
        if not len(captions):
            print(f"No images/captions in batch...")
            continue
        images = shard(images)
        encoded = p_encode(images, vqgan_params)
        encoded = encoded.reshape(-1, encoded.shape[-1])
        all_captions.extend(captions)
        all_encoding.extend(encoded.tolist())

        # save files
        if (idx + 1) % save_frequency == 0:
            print(f"Saving file {n_file}")
            batch_df = pd.DataFrame.from_dict(
                {"caption": all_captions, "encoding": all_encoding}
            )
            batch_df.to_parquet(f"{output_dir}/{n_file:03d}.parquet")
            all_captions = []
            all_encoding = []
            n_file += 1

    if len(all_captions):
        print(f"Saving final file {n_file}")
        batch_df = pd.DataFrame.from_dict(
            {"caption": all_captions, "encoding": all_encoding}
        )
        batch_df.to_parquet(f"{output_dir}/{n_file:03d}.parquet")

In [ ]:
encode_dataset(dl, output_dir=encoded_output, save_frequency=save_frequency)

----